In [0]:
# https://github.com/paraschopra/generating-text-small-corpus/blob/master/philosophy-generator-on-pretrained.ipynb

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57ba8000 @  0x7f2e92ba62a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [3]:
!pip install fastai

    100% |████████████████████████████████| 153kB 6.5MB/s 
    100% |████████████████████████████████| 112kB 29.1MB/s 
  Running setup.py bdist_wheel for bottleneck ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/f2/bf/ec/e0f39aa27001525ad455139ee57ec7d0776fe074dfd78c97e4
  Running setup.py bdist_wheel for nvidia-ml-py3 ... - done
  Stored in directory: /root/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7
Successfully built bottleneck nvidia-ml-py3


In [0]:
from fastai import *
from fastai.text import *

In [0]:
import pandas as pd
import numpy as np

In [7]:
cd 'gdrive/My Drive/app/shelbot/'

/content/gdrive/My Drive/app/shelbot


In [9]:
!ls

files+data  Shelbot.ipynb  sq.csv  UsingPreTrained.ipynb


In [10]:
df = pd.read_csv('sq.csv', sep=r'\<\|\>', header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [11]:
df.head()

,0
0,Interesting. You're afraid of insects and wome...
1,"Scissors cuts paper, paper covers rock, rock c..."
2,Why are you crying?
3,That's no reason to cry. One cries because one...
4,"For the record, it could kill us to meet new p..."


In [0]:
df[0] = df[0].str.lower()

In [14]:
df.head()

,0
0,interesting. you're afraid of insects and wome...
1,"scissors cuts paper, paper covers rock, rock c..."
2,why are you crying?
3,that's no reason to cry. one cries because one...
4,"for the record, it could kill us to meet new p..."


In [0]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [16]:
len(train_df), len(valid_df)

(8091, 426)

In [0]:
nan_rows = df[df[0].isnull()]

In [19]:
nan_rows

,0


In [0]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols=0)

In [28]:
torch.set_grad_enabled(False)

In [31]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.5)
with torch.enable_grad():
  learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.766544,4.294909,0.235817


In [32]:
learn.unfreeze()
with torch.enable_grad():
  learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.211672,3.937790,0.285280


In [0]:
wd=1e-7
lr=1e-3
lrs = lr

In [35]:
with torch.enable_grad():
  learn.fit(10,lrs, wd)

epoch,train_loss,valid_loss,accuracy
1,3.976359,3.802229,0.302455
2,3.871498,3.733679,0.309983
3,3.785404,3.695511,0.313672
4,3.711911,3.677020,0.313210
5,3.638432,3.587873,0.314898
6,3.563406,3.669779,0.314261
7,3.488365,3.686110,0.307912
8,3.405425,3.604347,0.309902
9,3.329803,3.674390,0.314289
10,3.258611,3.590584,0.314852


In [51]:
learn.predict("xxbos", n_words=50, temperature=0.75)

"xxbos then this is fun . xxbos you ca n't come . may i give you a spare ? xxbos i 've it out , so i 'm always here to express your hope that the best way to do that is to manipulate me . xxbos i 'm not going"

In [48]:
number_of_ideas = 10
ideas_counter = 0
all_ideas = []

for i in range(1000):
    idea = learn.predict("xxbos xxfld 1", n_words=20, temperature=0.8)
    print (idea)
    ideas = idea.split("xxbos xxfld 1")
    ideas = ideas[1:]
    print (ideas)
    if len(ideas) > 0:
      for idea in ideas:
          idea = idea.replace("xxbos xxfld 1 ","").strip()
          if idea:
              all_ideas.append(idea)
              ideas_counter = ideas_counter+1
    else:
      ideas_counter += 1
    print ('Counter: ' + str(ideas_counter))
    if ideas_counter > number_of_ideas:
        break

xxbos xxfld 1 , interesting . xxbos i do n't know . i was thinking of living with my sister in the airport
[" , interesting . xxbos i do n't know . i was thinking of living with my sister in the airport"]
Counter: 11


In [49]:
all_ideas

["that 's a joke . xxbos i asked that i should have someone to moment in . xxbos ah ,",
 "right , i appreciate why you are the only one who thinks you do n't need to understand the benefits",
 "it 's an emotion - loving parrot . that 's a shame . xxbos [ laughter ] and if you",
 "you know , i 'm going to tell you paper paper that i 've insane . xxbos does n't mean",
 "i 'm going to get your hair cut . xxbos so the next thing i 'm gon na need is",
 "with the first six weeks of our relationship , we can find out about what it 's like in a",
 "when you 're in a relationship , you 'd like to have dinner with us . xxbos oh , i",
 'the mini - pig and the dog being brought up in the wild . but now they have to cut',
 "disappointing . that 's not going to be mine . xxbos you know , i do n't know what i",
 "this is our first episode of star xxbos i 'm not sure your mom does get up . xxbos i",
 ", interesting . xxbos i do n't know . i was thinking of living with my sister in the airport"]